# VK API

VK API бесплатное, но нужно получить доступ


**Авторизация**

1. Перейти по ссылке https://vk.com/apps?act=manage
2. Создать новое приложение: standalone-приложение, название любое
3. Необходимо получить код подтверждения, может быть по номеру телефона
4. Если все ок, то вы перейдете в настройки приложения, там можно увидеть ключи доступа. На интересует **сервисный ключ доступа**

**Версии**

У API есть версии, поэтому вместе с токеном мы еще передаем информацию о том, какую версию мы используем. Версия отличает тем, какие параметры подаются, но больше это касается того, в каком виде мы получим ответ (структура ответа или наличие тех или иных полей может меняться от версии к версии).

In [29]:
import requests
from tqdm.auto import tqdm
from datetime import datetime
import pandas as pd

Зафиксируем версию и токен в константах. В качестве токена подставляется сервисный ключ из настроек приложения. Укажем ту, которая дефолтно показывается в документации.

In [3]:
TOKEN = ""
VERSION = "5.130"

## Метод wall.get

Первое, что мы сделаем - попробуем скачать информацию со стены, чтобы научиться собирать текстовые данные.

Подробная документация по ссылке: https://vk.com/dev/wall.get. Тут можно посмотреть на пример запроса, описание полей.

In [4]:
wall_get_url = "https://api.vk.com/method/wall.get" # endpoint, на который мы отправляем такие запросы

Запрашиваем 2 последних поста со страницы юзера с id = 1 (Павел Дуров). 

Для сообществ ID будут отрицательными (например, -1)

In [5]:
data = requests.get(
    wall_get_url, 
    params={
        "owner_id": 1,  # ID юзера
        "count": 2,  # кол-во постов
        "v": VERSION, # версия API
        "access_token": TOKEN  # токен доступа
    }
).json()

Мы получим ответ, который представляет собой словарь, где по ключу response лежит сам ответ.

Внутри лежит параметр count с числом записей (всего). В items сами посты (2, как мы просили).

Для каждого поста есть информация по объекту post. Подробное описание на странице https://vk.com/dev/objects/post

In [6]:
data

{'response': {'count': 296,
  'items': [{'id': 2442097,
    'from_id': 1,
    'owner_id': 1,
    'date': 1525805964,
    'post_type': 'post',
    'text': 'Иногда говорят, что Telegram был заблокирован в России, так как “закон есть закон”. Однако Telegram заблокирован в России как раз вопреки главному закону страны – Конституции. Решения судов и законы, противоречащие Конституции, не имеют силы. А это значит, что и сама блокировка Telegram незаконна. \n\nЕсли бы ФСБ ограничилась запросом информации о нескольких террористах, то ее требование вписывалось бы в рамки Конституции. Однако речь идет о передаче универсальных ключей шифрования с целью последующего бесконтрольного доступа к переписке неограниченного круга лиц. A это – прямое нарушение 23-й статьи Конституции о праве каждого на тайну переписки.\n\nПо этой причине юристы из “Агоры” сегодня обжаловали решение Верховного суда России о законности приказа ФСБ. Надеюсь, власти России откажутся от языка неисполнимых ультиматумов, на кото

Можно заметить, что дата отображается в виде числа. Это специальный формат unixtimestamp, который очень часто используется, так как целые числа - это универсальный способ хранения, который можно исопльзовать в любой системе (JSON, любые БД и прочие)

In [10]:
unixtime = data['response']['items'][0]['date']
utc = datetime.fromtimestamp(unixtime)
print(unixtime, utc)

1525805964 2018-05-08 21:59:24


## wall.getComments

Следующий метод - это сбор комментариев к посту, что тоже может быть интересно для изучения диалогов или чего-то подобного.

Основные параметры: owner_id, post_id. Их мы можем достать из информации о постах на стене.

Можно это сделать и вручную, например, если открыть пост во всплывающем окне, то по адресу в адресной строке можно понять эти id.

https://vk.com/id1?w=wall1_2442097 : owner_id = 1, post_id = 2442097

Для примера возьмем пост из СМИ, где можно оставлять комментарии.

In [13]:
get_comments_url = "https://api.vk.com/method/wall.getComments"

In [16]:
data = requests.get(
    get_comments_url, 
    params={
        "owner_id": -76982440,
        "post_id": 5011073,
        "count": 2,
        "need_likes": 1,
        "v": VERSION,
        "access_token": TOKEN
    }
).json()

In [17]:
data

{'response': {'count': 16,
  'items': [{'id': 5011074,
    'from_id': 432646319,
    'post_id': 5011073,
    'owner_id': -76982440,
    'parents_stack': [],
    'date': 1618533695,
    'text': 'Страна советов',
    'likes': {'count': 14, 'user_likes': 0, 'can_like': 1},
    'thread': {'count': 0,
     'items': [],
     'can_post': True,
     'show_reply_button': True,
     'groups_can_post': True}},
   {'id': 5011078,
    'from_id': 174352380,
    'post_id': 5011073,
    'owner_id': -76982440,
    'parents_stack': [],
    'date': 1618533953,
    'text': 'Красавчик 👍🏼❤️',
    'likes': {'count': 0, 'user_likes': 0, 'can_like': 1},
    'thread': {'count': 0,
     'items': [],
     'can_post': True,
     'show_reply_button': True,
     'groups_can_post': True}}],
  'current_level_count': 10,
  'can_post': True,
  'show_reply_button': True,
  'groups_can_post': True}}

## groups.getMembers

Можно получить информацию о пользователях, например, о тех, кто подписан на опредленную группу.

Попробуем еще параметр offset, который может пригодиться, когда данных много и придется скачивать "страницы" или блоки по 100 или 1000 объектов (1000 - ограничение для этого метода)

In [18]:
group_members = "https://api.vk.com/method/groups.getMembers"

In [20]:
group = "dormitory8hse"

In [21]:
data = requests.get(
    group_members,
    params={
        'group_id': group,
        'access_token': TOKEN,
        'v': VERSION,
        'offset': 0
    }
).json()

Кол-во объектов всего

In [23]:
data["response"]["count"]

5884

В качестве ответа по людям - просто список ID. По ним уже дальше можно запрашивать подробную информацию о пользователях.

In [24]:
data["response"]["items"][:10]

[11952, 20090, 56613, 62028, 80420, 81206, 96206, 97723, 113393, 144980]

А теперь следующая страница. Это можно делать в цикле, чтобы выкачать всех

In [25]:
data = requests.get(
    group_members,
    params={
        'group_id': group,
        'access_token': TOKEN,
        'v': VERSION,
        'offset': 1000
    }
).json()

data["response"]["items"][:10]

[23487241,
 23498897,
 23540705,
 23555478,
 23665918,
 23757830,
 23762493,
 23783183,
 23795292,
 23804422]

## users.get

Попробуем собрать информацию о тех, кто попал

In [26]:
users_get_url = "https://api.vk.com/method/users.get"

In [39]:
all_users_data = []

for user in tqdm(data["response"]["items"][:500]):
    user_info = requests.get(
        users_get_url,
        params={
            'user_ids': user,
            'fields': 'bdate,home_town,universities',
            'access_token': TOKEN,
            'v': VERSION
        }
    ).json()
    all_users_data.extend(user_info["response"])

In [40]:
df = pd.DataFrame(all_users_data)

# чтобы не показывать в открытом месте данные, возьмем для демонстрации толькодва столбца
df = df.dropna(subset=["home_town", "universities"])
df = df[["home_town", "universities"]] 

In [41]:
df.head()

,home_town,universities
1,Краснодар,[]
5,станица Казанская,"[{'chair': 6900, 'chair_name': 'Квантовой ради..."
6,Казань,"[{'chair': 2037657, 'chair_name': 'Прикладная ..."
23,Тамбов,"[{'city': 1, 'country': 1, 'id': 128, 'name': ..."
31,Киев,"[{'chair': 38286, 'chair_name': 'Информационны..."


Университеты нужно еще дальше разбирать, а вот города можно подсчитать сразу

In [42]:
df["home_town"].value_counts()

                               29
Омск                            5
Красноярск                      3
Тверь                           3
Уфа                             3
                               ..
Кишинев                         1
Асбест                          1
Ижевск                          1
Минск                           1
Нижний Новгород и Чебоксары     1
Name: home_town, Length: 64, dtype: int64

Видно, что не все идеально (есть сразу несколько), но после препроцессинга может быть ок.

## Задание

### 1

Попробовать собрать данные участников одной из групп и подсчитать статистику по какому-либо параметру (город, год рождения или иное).

1. Выбираем группу
2. Делаем пробный запрос
3. Пишем цикл по сбору (установка offset +1000 каждый раз)
4. Сохраняем ID
5. В цикле получаем информацию по пользователям
6. Сохраняем в датафрейм
7. Считаем статистику (и делаем препроцессинг, если нужно)

### 2

Собрать посты и/или комментарии, где авторы разные и попробовать сравнить употребление какого-либо слова у двух групп пользователей (Москва / не-Москва, Петербург / Москва, любые другие). Возможно, данных нужно будет много, но можно попробовать популярное слово. Главное - попробовать сделать основу для решения подобной задачи.

1. Выбираем группу
2. Собраем посты, сохраняем.
3. Для каждого поста собираем комментарии
4. По всем объектам собираем уникальные айди пользователей (лучше в порядке убывания частоты появления
5. Собираем информацию по ним, выделяем наши группы
6. Лемматизируем (если надо)
7. Считаем стастику